# <a id="home">Table of Content</a>
1. [Studi kasus part I](#studi-kasus-i)
2. [Studi kasus part II](#studi-kasus-ii)

# <a id="studi-kasus-i">Studi Kasus part I: Training & Fine-tuning Decision Trees</a> (Back to [Home](#home))

Dalam Part I ini, anda akan melatih dan _fine-tune_ pohon keputusan untuk dataset moons. 

<div>
    <img src="images/decision_trees/moons-dataset.png" width=500/>
</div>
<center>
Contoh dataset `moons`
</center>    

Mari _generate_ dataset moons dengan menggunakan `make_moons(n_samples=10000, noise=0.4)`.

Tambahkan juga `random_state=42`:

In [1]:
#tata
from sklearn.datasets import make_moons

X_moons, y_moons = make_moons(n_samples=10000, noise=0.4, random_state=42)

Mari kita pisahkan dataset menjadi **train set** dan **test set** dengan menggunakan `train_test_split()`.    
Buatlah _test set_ sebesar 20% dari _dataset_.    
Gunakan `random_state=42`.

In [2]:
#tata
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_moons, y_moons, test_size = 0.2, random_state = 42)

Gunakan _grid search_ dikombinasikan _cross-validation_ dengan menggunakan kelas `GridSearchCV` untuk mencari _hyperparameter_ yang bagus untuk `DecisionTreeClassifier`.    
   
Berikut nilai-nilai _hyperparameter_ yang hendak di-_fine-tuning_:   
- $2 \leq \texttt{max_leaf_nodes} \leq 99$ 
- $1 \leq \texttt{max_depth} \leq 6$
- $2 \leq \texttt{min_samples_split} \leq 4$ 

dengan masing-masing `max_leaf_nodes`, `max_depth`, dan `min_samples_split` merupakan bilangan bulat.
   
**Catatan**: Gunakan 3-fold CV dan `random_state=42`.   

In [3]:
#thor
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

params = {
    
    'max_leaf_nodes': list(range(2, 99)),
    'max_depth': list(range(1, 6)),
    'min_samples_split': list(range(2, 4))
}

DecTree = DecisionTreeClassifier()

grid_search_cv = GridSearchCV(DecTree, param_grid=params, cv = 3, scoring='accuracy')

Mari kita latih `grid_search_cv` pada `X_train` dan `y_train`.

In [4]:
#thor
grid_search_cv.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [1, 2, 3, 4, 5],
                         'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_split': [2, 3]},
             scoring='accuracy')

Mari kita tampilkan **best hyperparameter** dari _decision tree_ tersebut.

In [5]:
#bobi
print("Grid Search Hasil:")
print(f"Parameter Terbaik: {grid_search_cv.best_params_}")
print(f"Akurasi Terbaik: {round(grid_search_cv.best_score_*100,2)}%")

Grid Search Hasil:
Parameter Terbaik: {'max_depth': 2, 'max_leaf_nodes': 4, 'min_samples_split': 2}
Akurasi Terbaik: 85.25%


Sekarang mari kita latih _decision tree_ dengan seluruh _train set_ kemudian kita ukur performance _decision tree_ pada **test set**.    
   
Berapakah akurasi yang anda peroleh?

By default, `GridSearchCV` melatih _best model_ yang sudah ditemukan pada keseluruhan train set.   
Jadi kita tidak perlu melatih model _decision tree_ dari _scratch_.    
Kita dapat mengevaluasi keakuratan model sbb:

In [6]:
#bobi
from sklearn.metrics import accuracy_score

best_model = grid_search_cv.best_estimator_
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Akurasi best model: {round(accuracy*100,2)}%")

Akurasi best model: 86.3%


# <a id="studi-kasus-ii">Studi Kasus part II: Menumbuhkan Hutan (_Forest_)</a> (Back to [Home](#home))

## Prelude to Random Forest $\Longrightarrow$ one of the most powerful ML models

Mari kita lanjutkan studi kasus part I dengan meng-_generate_ 1.000 (seribu) _subsets_ dari _train set_, setiap _subset_ mengandung 100 _instances_ yang dipilih secara acak.    

**Hint**: kita gunakan Scikit-Learn's `ShuffleSplit` class untuk melakukan ini.

In [7]:
#dapid
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances,
                  random_state=42)

for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

Terdapat 1000 _mini set_ atau _subset_ dari _train set_.

In [8]:
#dapit
len(mini_sets)

1000

Setiap _subset_ terdiri dari 2 bagian, yaitu: `X_mini_train` dan `y_mini_train`.

In [9]:
#dapit
len(mini_sets[0])

2

Setiap `X_mini_train` mempunyai 100 _instances_.     
Setiap `y_mini_train` mempunyai 100 _instances_.     

In [10]:
#wili
len(mini_sets[0][0]), len(mini_sets[0][1])

(100, 100)

- Sekarang mari kita _train_ satu pohon keputusan untuk setiap _subset_ dengan _best hyperparameter_ yang diperoleh dari Studi Kasus part I.    
- Evaluasi 1000 pohon keputusan ini pada _test set_.    
- Karena pohon keputusan-pohon keputusan ini dilatih pada _dataset_ yang lebih kecil (_subset_), _performance_ pohon keputusan-pohon keputusan ini **lebih jelek daripada pohon keputusan di Studi Kasus part I**, yaitu mencapai akurasi hanya sekitar 80\%.

In [11]:
#wili
from sklearn.base import clone
import numpy as np

forest = [clone(grid_search_cv.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    #### Write Your Code Here ####
    tree.fit(X_mini_train, y_mini_train)
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    
    
    accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

0.831607

_Here comes the magic!_    
   
Untuk setiap _instance_ _test set_, kita _generate_ prediksi dari 1000 (seribu) pohon keputusan. 

In [12]:
#wili
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test) #### Write Your Code Here ####

Sekarang hitunglah prediksi yang sering muncul (**modus**) dari 1000 pohon keputusan tersebut (anda dapat menggunakan SciPy's `mode()` function untuk menghitung **modus**).    
   
**Modus** memberikan kita prediksi dengan **voting suara terbanyak** atas _test set_.   

Mari kita hitung modus dengan cara memanggil fungsi `mode` pada `Y_pred` dengan diikuti oleh `axis=0` dan `keepdims=False`.

In [13]:
#arth
from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0, keepdims=False) #### Write Your Code Here ####

Terakhir, mari kita evaluasi prediksi pada _test set_ ini dengan menggunakan fungsi `accuracy_score`.

In [14]:
#arth
#### Write Your Code Here ####
print("Accuracy score : ", accuracy_score(y_pred_majority_votes, y_test))

Accuracy score :  0.869


Seharusnya anda akan memperoleh akurasi yang **sedikit tinggi** dari akurasi model pertama anda di Studi Kasus part I. Peningkatan yang terjadi adalah sebesar 0.5 sampai dengan 1.5%.    
   
Selamat! Anda sudah melatih _classifier_ **Random Forest**.   

<center>
    <h1>The End</h1>
</center>